In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = False
is_export = False

landmarks_path = '../data/landmarks/refine_6kmh_braless_18markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../data/test/braless_random_landmarks.pkl'

start=0
stride = 12
end=120

export_folder = 'output/rbf_ablation/'

# Data Loading

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=12, contour=contour, clip_bound='y', margin=0.5)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

# Ablation testing class

In [ ]:
from mesh4d.regist.reg_rbf import Obj4d_RBF, Trans_Nonrigid_RBF

class Obj4d_RBF_ablation(Obj4d_RBF):
    def __init__(self, enable_rigid: bool = False, enable_nonrigid: bool = False, k_nbr=1, kernel='thin_plate_spline', **kwargs):
        super().__init__(enable_rigid, enable_nonrigid, **kwargs)
        self.k_nbr = k_nbr
        self.kernel = kernel

    def process_nonrigid_dynamic(self, idx_source: int, idx_target: int, landmark_name: str, **kwargs):
        trans = Trans_Nonrigid_RBF(
            source_obj=self.obj_ls[idx_source],
            target_obj=self.obj_ls[idx_target],
        )
        trans.regist(landmark_name, k_nbr=self.k_nbr, kernel=self.kernel, **kwargs)
        self.obj_ls[idx_source].set_trans_nonrigid(trans)

# Kernel selection

In [ ]:
kernel_results = {
    'control landmarks': {},
    'non-control landmarks': {},
}

## Control landmarks

In [ ]:
import time
import mesh4d
from mesh4d import kps
mesh4d.output_msg = False

for k_nbr in [0, ]:
    for kernel in ['thin_plate_spline', 'linear', 'cubic', 'quintic']:
        kernel_results['control landmarks'][kernel] = {}
        results = kernel_results['control landmarks'][kernel]
        
        print('='*70)
        print('knbr {} - kernel {}'.format(k_nbr, kernel))

        # registration
        start_time = time.time()

        o4 = Obj4d_RBF_ablation(
            fps=120 / stride,
            enable_rigid=False,
            enable_nonrigid=True,
            k_nbr=k_nbr,
            kernel=kernel,
        )

        o4.add_obj(*body_ls)
        o4.load_markerset('landmarks', landmarks)
        o4.regist('landmarks')

        duration = time.time() - start_time
        
        # computation time
        print("computation time: {}".format(duration))
        results['duration'] = duration
        
        # control landmarks
        kps_source = landmarks.get_time_coord(0)
        o4.vkps_track(kps_source, frame_id=0)
        vkps = o4.assemble_markerset(name='vkps')
        diff = kps.MarkerSet.diff(vkps, landmarks)

        print(diff['diff_str'])
        results['dist_mean'] = diff['dist_mean']
        results['dist_std'] = diff['dist_std']

## Non-control landmarks

In [ ]:
import mesh4d
import numpy as np
mesh4d.output_msg = False

for k_nbr in [0, ]:
    for kernel in ['thin_plate_spline', 'linear', 'cubic', 'quintic']:
        kernel_results['non-control landmarks'][kernel] = {}
        results = kernel_results['non-control landmarks'][kernel]

        print('='*70)
        print('knbr {} - kernel {}'.format(k_nbr, kernel))

        dist_ls = []
        duration_ls = []

        # k-fold cross-verification
        for name in landmarks.markers.keys():
            # split dataset
            landmarks_test, landmarks_train = landmarks.split((name, ))

            # registration
            start_time = time.time()
            
            o4 = Obj4d_RBF_ablation(
                fps=120 / stride,
                enable_rigid=False,
                enable_nonrigid=True,
                k_nbr=k_nbr,
                kernel=kernel,
            )

            o4.add_obj(*body_ls)
            o4.load_markerset('landmarks_train', landmarks_train)
            o4.load_markerset('landmarks_test', landmarks_test)
            o4.regist('landmarks_train')

            duration = time.time() - start_time
            duration_ls.append(duration)

            # virtual key points tracking
            kps_source = landmarks_test.get_time_coord(0)
            o4.vkps_track(kps_source, frame_id=0, name='vkps')
            vkps = o4.assemble_markerset(name='vkps')
            diff = kps.MarkerSet.diff(vkps, landmarks_test)

            for marker_diff in diff['diff_dict'].values():
                dist_ls.append(marker_diff['dist'])
        
        dist_mean = np.mean(np.array(dist_ls))
        dist_std = np.std(np.array(dist_ls))

        print("computation time: {}".format(np.mean(np.array(duration))))
        print('overall error: {:.2f} ± {:.2f} (mm)'.format(dist_mean, dist_std))

        results['duration'] = duration
        results['dist_mean'] = diff['dist_mean']
        results['dist_std'] = diff['dist_std']

In [ ]:
utils.save_pkl_object(kernel_results, export_folder, 'kernel_results')

# Post-alignment

In [ ]:
align_results = {
    'merge': {},
}

## Shape mergence improvement

In [ ]:
import mesh4d
import numpy as np
from scipy.spatial import KDTree
mesh4d.output_msg = False

for k_nbr in [0, 1, 2, 5]:
    for kernel in ['thin_plate_spline', ]:
        align_results['merge'][k_nbr] = {}
        results = align_results['merge'][k_nbr]

        print('='*70)
        print('knbr {} - kernel {}'.format(k_nbr, kernel))

        # registration
        start_time = time.time()

        o4 = Obj4d_RBF_ablation(
            fps=120 / stride,
            enable_rigid=False,
            enable_nonrigid=True,
            k_nbr=k_nbr,
            kernel=kernel,
        )

        o4.add_obj(*body_ls)
        o4.load_markerset('landmarks', landmarks)
        o4.regist('landmarks')

        duration = time.time() - start_time
        
        # computation time
        print("computation time: {}".format(duration))
        results['duration'] = duration
        
        # shape mergence
        dist_ls = []

        for id in range(len(o4.obj_ls) - 1):
            # get predicted deform points
            # source_points = o4.obj_ls[id].get_vertices()
            source_points = breast_ls[id].get_vertices()
            deform_points = o4.obj_ls[id].trans_nonrigid.shift_points(source_points)
            
            # search deform points' nearest points in target points
            target_points = o4.obj_ls[id + 1].get_vertices()
            tree = KDTree(target_points)
            _, idx = tree.query(deform_points)
            target_nearest_points = target_points[idx]

            # calculate the distance from deform points and its nearest points in target points
            dist = np.linalg.norm(target_nearest_points - deform_points, axis=1)
            dist_ls.append(dist)

        dist_array = np.concatenate(dist_ls)
        results['dist_mean'] = np.mean(dist_array)
        results['dist_std'] = np.std(dist_array)

        print("nearest point distance: {:.2f} ± {:.2f} (mm)".format(results['dist_mean'], results['dist_std']))

In [ ]:
utils.save_pkl_object(align_results, export_folder, 'align_results')